#  AE Modeling: Script 1

In [1]:
import pandas as pd
import numpy as np
import os
import json
import pickle
from sklearn.model_selection import train_test_split

## Download data from s3 bucket

In [7]:
! aws s3 cp s3://cmsai-mrk-amzn/CSVModelInputs/miguel/365-wo-death/ data/ae_raw/ --exclude "*" --include "ae_patients_365_2011*.csv" --recursive  # these files are death-removed

download: s3://cmsai-mrk-amzn/CSVModelInputs/miguel/365-wo-death/ae_patients_365_20110301.csv to data/ae_raw/ae_patients_365_20110301.csv
download: s3://cmsai-mrk-amzn/CSVModelInputs/miguel/365-wo-death/ae_patients_365_20110201.csv to data/ae_raw/ae_patients_365_20110201.csv
download: s3://cmsai-mrk-amzn/CSVModelInputs/miguel/365-wo-death/ae_patients_365_20110501.csv to data/ae_raw/ae_patients_365_20110501.csv
download: s3://cmsai-mrk-amzn/CSVModelInputs/miguel/365-wo-death/ae_patients_365_20110101.csv to data/ae_raw/ae_patients_365_20110101.csv
download: s3://cmsai-mrk-amzn/CSVModelInputs/miguel/365-wo-death/ae_patients_365_20110401.csv to data/ae_raw/ae_patients_365_20110401.csv
download: s3://cmsai-mrk-amzn/CSVModelInputs/miguel/365-wo-death/ae_patients_365_20110601.csv to data/ae_raw/ae_patients_365_20110601.csv
download: s3://cmsai-mrk-amzn/CSVModelInputs/miguel/365-wo-death/ae_patients_365_20110701.csv to data/ae_raw/ae_patients_365_20110701.csv
download: s3://cmsai-mrk-amzn/CSVM

In [10]:
!aws s3 cp s3://cmsai-mrk-amzn/CSVModelInputs/test_ae_patients_365_20111201.csv data/ae_raw/ae_patients_365_20111201.csv

download: s3://cmsai-mrk-amzn/CSVModelInputs/test_ae_patients_365_20111201.csv to data/ae_raw/ae_patients_365_20111201.csv


In [2]:
# Define the columns 
x_lst = [str(x) for x in range(365,-1,-1)]
y_lst = ['d_5990', 'd_78605', 'd_486', 'd_78650', 'd_78079', 'd_78900', 'd_78609', 'd_7862', 'd_1101',
         'd_78701', 'd_5789', 'd_78791', 'd_6826', 'd_78659', 'd_78907',
         'd_7840', 'd_28860', 'd_4660', 'd_6829', 'd_00845']

## Build vocab of events

In [8]:
import model_utils
from collections import Counter
import numbers
import time
import torch
import numbers
from collections import Counter
import torch
import torch.functional as F
from torch.nn.utils.rnn import pad_sequence
from torchtext.datasets import text_classification
from torchtext.vocab import Vocab

import metrics
import numpy as np

def build_vocab(df, feat_colnames, min_freq=1, specials=['<pad>', '<unk>'], specials_first=True):
    '''
    Create a vocabulary to be used with Script2. This maps all events to an index, including <pad> and <unk> and
    nan, which represents padding of sentences, unknown events, and no events respectively
    '''
    def build_counter(df, feat_colnames):
        counter = Counter()
        words = df[feat_colnames].values.ravel('K')
        print("start word number: ", words.shape)
        new_words = []
        for x in words:
            x = str(x).replace('d_s', 'd_')
            new_words.extend(x.split(', '))
        print("exact word number: " , len(new_words))
        counter.update(new_words)
        if not isinstance(min_freq, numbers.Number):
            raise ValueError(f'Something wrong with {min_freq}')
        return counter
  
    print("df shape: ", df.shape)
    positive_df = df[(df[y_lst] != 0).any(1)]
    print("postive shape: ", positive_df.shape)
    pos_counter = build_counter(positive_df, feat_colnames)
    print("nan number: ", pos_counter['nan'])

    vocab = Vocab(pos_counter, min_freq=min_freq, specials=specials, specials_first=specials_first)

    print('Completed vocabulary')
    return vocab

In [3]:
import model_utils
from collections import Counter
import numbers
import time
import torch
import numbers
from collections import Counter
import torch
import torch.functional as F
from torch.nn.utils.rnn import pad_sequence
from torchtext.datasets import text_classification
from torchtext.vocab import Vocab

import metrics
import numpy as np

counter = Counter()

def build_list(df, feat_colnames):
    words = df[feat_colnames].values.ravel('K')
    print("start word number: ", words.shape)
    new_words = []
    for x in words:
        x = str(x).replace('d_s', 'd_').replace(' ', '')
        new_words.extend(x.split(','))
    print("exact word number: " , len(new_words))
    return new_words
    
def build_vocab_mp(month):
    if month < 10:
        m = '0' + str(month)
    else:
        m = str(month)
    FP = 'data/ae_raw/ae_patients_365_2011' + m + '01.csv'
    ae_targets_365_df = pd.read_csv(FP)
    positive_df = ae_targets_365_df[(ae_targets_365_df[y_lst] != 0).any(1)]
    print("postive shape: ", positive_df.shape)
    return build_list(positive_df, x_lst[-120:])

# import multiprocessing
# from multiprocessing import Pool
# num_workers = multiprocessing.cpu_count()
# with Pool(processes=num_workers) as pool:
#     words = pool.map(build_vocab_mp, list(range(1, 13)))
    
# for word in words:
#     counter.update(word)
# print(len(counter))
# vocab = Vocab(counter, min_freq=1, specials=['<pad>', '<unk>'], specials_first=True)


In [9]:
torch.save(vocab, 'data/ae_pos_vocab_last120_whole_non3')

In [24]:
# read 11 months data and generate vocab
for i in range(2, 12):
    if i < 10:
        m = '0' + str(i)
    else:
        m = str(i)
    FP = 'data/ae_raw/ae_patients_365_2011' + m + '01.csv'
    ae_targets_365_df = pd.read_csv(FP)
    positive_df = ae_targets_365_df[(ae_targets_365_df[y_lst] != 0).any(1)]
    print("postive shape: ", positive_df.shape)
    counter.update(build_list(positive_df, x_lst[-90:]))    
vocab = Vocab(counter, min_freq=1, specials=['<pad>', '<unk>'], specials_first=True)
torch.save(vocab, 'data/ae_pos_vocab_last90_whole_non3')

/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (2,3,5,6,7,8,9,10,11,13,14,15,17,19,20,21,22,23,24,25,26,27,29,30,31,32,33,34,35,36,37,38,40,41,42,43,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,107,108,109,110,111,112,113,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,131,132,133,135,136,137,138,139,140,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,164,165,167,168,169,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,192,193,194,195,196,198,200,201,202,203,204,205,206,207,208,209,210,211,212,214,216,217,218,219,220,223,224,225,226,227,228,230,231,232,233,237,238,241,242,243,244,245,246,247,248,249,251,252,253,254,256,257,258,259,260,262,264,265,266,267,268,269,271,272,273,275,276,27

month size:  768843225
99879
(1886796, 387)
postive shape:  (198397, 387)
start word number:  (17855730,)
exact word number:  24615926


/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (1,3,5,6,7,8,9,11,12,13,14,15,16,17,19,20,21,24,25,26,27,28,30,32,33,34,35,36,38,39,40,41,42,43,44,45,46,47,48,49,50,51,53,54,55,56,57,59,60,61,62,63,64,66,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,86,87,88,89,90,91,92,95,96,97,98,100,101,102,103,104,105,106,107,108,110,111,112,113,116,117,118,119,120,122,123,125,126,127,128,129,130,131,132,133,134,138,139,140,141,142,143,144,145,146,147,148,149,150,152,153,154,155,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,186,187,188,189,190,191,192,193,194,196,200,202,203,204,207,208,209,210,213,214,216,218,220,222,223,224,226,227,230,231,236,237,238,239,240,241,242,243,244,246,247,248,249,250,251,252,253,254,255,256,257,258,259,261,263,264,265,266,269,270,271,272,273,274,275,276,277,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,2

month size:  763305255
100453
(1871912, 387)
postive shape:  (197781, 387)
start word number:  (17800290,)
exact word number:  25243592


/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (3,4,6,8,10,11,14,16,18,20,21,23,24,25,27,28,29,31,32,33,35,39,41,42,44,45,46,47,49,51,52,53,58,59,60,61,62,64,65,66,67,68,70,71,73,74,75,77,79,80,81,85,86,87,88,89,91,92,93,94,95,96,97,98,101,102,103,105,106,108,109,111,112,113,114,115,116,119,120,121,122,123,124,126,127,128,129,130,131,134,135,136,137,138,139,140,141,143,144,145,146,150,151,152,155,158,159,164,165,167,170,171,172,173,175,178,179,183,186,193,195,200,206,207,208,212,213,214,220,221,224,225,227,228,231,232,233,234,235,237,238,239,240,241,242,243,244,245,246,247,248,249,251,252,254,255,256,260,261,262,263,266,267,268,269,270,272,273,275,276,277,278,280,282,283,284,285,286,287,289,290,291,293,295,296,297,298,299,300,301,303,304,305,306,307,308,310,311,312,313,314,315,316,317,318,319,320,321,322,324,325,326,327,328,329,331,332,333,339,340,341,342,343,344,345,346,347,350,352,353,354,355,357,358,

...

month size:  727565418
101340
(1778674, 387)
postive shape:  (180554, 387)
start word number:  (16249860,)
exact word number:  23898030


/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (10,23,24,37,42,45,51,52,56,58,59,65,66,69,72,73,79,80,85,86,87,92,93,94,98,101,107,108,114,115,122,125,128,129,135,136,137,142,143,149,150,153,155,156,157,161,164,170,171,176,177,178,184,185,191,192,198,199,201,205,206,211,212,213,217,219,220,225,226,227,233,234,239,240,241,242,247,248,252,254,255,259,260,261,262,265,267,268,269,275,276,277,283,289,290,293,296,297,303,304,307,309,310,311,317,318,324,325,331,332,338,339,340,346,353,360) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


month size:  718489881
101556
(1755007, 387)
postive shape:  (185024, 387)
start word number:  (16652160,)
exact word number:  24476747


/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (7,14,20,25,27,28,35,42,48,49,54,55,56,62,63,69,70,76,77,84,90,91,94,97,98,102,103,104,105,107,110,111,112,117,118,119,120,124,125,126,132,133,139,140,145,146,147,150,151,153,154,157,160,161,167,168,170,174,175,180,181,182,184,188,189,195,196,202,203,207,209,210,211,216,217,220,223,224,228,229,230,231,234,237,238,239,244,245,246,251,252,258,259,263,265,266,273,277,279,280,286,287,293,294,300,301,308,309,315,321,322,329,336,350,356,357,364,365) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


month size:  708178266
101445
(1728473, 387)
postive shape:  (183489, 387)
start word number:  (16514010,)
exact word number:  24382711


In [26]:
print(len(vocab))

28362


## Build dataset and save for script 2

In [3]:
# Define the columns 
x_lst = [str(x) for x in range(365,-1,-1)]
y_lst = ['d_5990', 'd_78605', 'd_486', 'd_78650', 'd_78079', 'd_78900', 'd_78609', 'd_7862', 'd_1101',
         'd_78701', 'd_5789', 'd_78791', 'd_6826', 'd_78659', 'd_78907',
         'd_7840', 'd_28860', 'd_4660', 'd_6829', 'd_00845']

In [4]:
import time
import random
def build_dataset(df, vocab, feat_colnames, label_colnames, day_length=90, max_length=30, max_sentence_length=500):
    '''
    Subsets the entire dataset into a dataset to be used later.
    - Specific vocabulary
    - By number of days (whole dataset is 365)
    
    Returns a list of data and attributes needed by Script 2: patientid_dischargeid key, sequence of events, targets, and 
    mask (identifying padded regions)
    '''
    start_time = time.time()
    print("used days: ", feat_colnames[-day_length], feat_colnames[-1])
    
    data = df[feat_colnames[-day_length:]].to_numpy()
    labels = df[label_colnames].to_numpy()
    
    count = 0
    sequence = []
    valid_id = []
    pad_mask = []

    print("total size before: ", data.shape)
    for i in range(len(data)):
        sentence = []
        mask = []
        pos = []
        for j in range(len(data[i])-1, -1, -1):
            words = str(data[i][j])
            if words == 'nan':
                continue
            words = words.replace('d_s', 'd_').replace(' ', '').split(', ')
            words = sorted([vocab.stoi[w] if w in vocab.stoi else vocab.stoi['<unk>'] for w in words])
            
            if len(words) > max_length:
                words = words[:max_length]
        
            
            sentence = words + sentence

            if len(sentence) > max_sentence_length:
                sentence = sentence[-max_sentence_length:]
                break
                
        if len(sentence) == 0:
            if labels[i].any():
                count += 1
            continue
            
        valid_id.append(i)
        pad_l = (max_sentence_length - len(sentence))
        mask = [1] * len(sentence) + [0] * pad_l
        sentence = sentence + [vocab.stoi['<pad>']] * pad_l
        sequence.append(sentence)
        pad_mask.append(mask)
        
    finish_time = time.time()
    
    print('New dataset created')
    print("sequence length: ", len(sequence))
    print("empty events with nonzero labels: ", count)
    
    labels = labels[valid_id]
    patient_ids = df['patient_id'].to_numpy()[valid_id]
    pad_mask = np.array(pad_mask)
    sequence = np.array(sequence)
    print("time: ", finish_time - start_time)
    
    return [patient_ids, sequence, labels, pad_mask]



In [5]:
vocab = torch.load('data/ae_pos_vocab_last120_whole_non3')

In [11]:
import multiprocessing
from multiprocessing import Pool
num_workers = multiprocessing.cpu_count()

def generate_procdata(m):
    if m < 10:
        m = '0' + str(m)
    else:
        m = str(m)
    FP = 'data/ae_raw/ae_patients_365_2011' + m + '01.csv'
    ae_targets_365_df = pd.read_csv(FP)
    print("month size: ", str(m), ae_targets_365_df.size)
    
    whole_dataset = None
    whole_dataset = build_dataset(ae_targets_365_df, vocab, x_lst, y_lst, day_length=120, max_length=30, max_sentence_length=500)
    pickle.dump(whole_dataset, open("data/ae_process/np_ae_last120_vocab120_non3digit_month"  + m + ".pkl", 'wb'), protocol=4)  
    
with Pool(processes=num_workers) as pool:
    pool.map(generate_procdata, [12])

/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/multiprocessing/process.py:93: DtypeWarning: Columns (32,33,39,40,46,47,53,54,58,60,61,63,64,67,68,74,75,80,81,82,88,89,95,96,102,103,108,109,110,114,116,117,123,124,127,128,130,131,137,138,143,144,145,150,151,152,157,158,159,165,166,171,172,173,179,180,181,186,187,193,194,199,200,201,204,207,208,209,214,215,216,218,221,222,228,229,235,236,242,243,245,249,250,256,257,259,263,264,268,269,270,271,277,278,279,284,285,291,292,294,298,299,305,306,313,319,320,327,333,334,340,341,347,348,354,355,358,359,360,361,362) have mixed types.Specify dtype option on import or set low_memory=False.
  self._target(*self._args, **self._kwargs)


month size:  12 749593458
used days:  119 0
total size before:  (1936934, 120)
New dataset created
sequence length:  1516165
empty events with nonzero labels:  5818
time:  225.47493076324463


In [ ]:
print(type(words))

In [10]:
# generate the whole data from 11 months
for i in [1, 2, 3,4,5,6,7,8,9,10,11]:
    if i < 10:
        m = '0' + str(i)
    else:
        m = str(i)
    
    FP = 'data/ae_raw/ae_patients_365_2011' + m + '01.csv'
    ae_targets_365_df = pd.read_csv(FP)
    print("month size: ", str(i), ae_targets_365_df.size)
    
    whole_dataset = None
    whole_dataset = build_dataset(ae_targets_365_df, vocab, x_lst, y_lst, day_length=120, max_length=30, max_sentence_length=500)
    pickle.dump(whole_dataset, open("data/ae_process/np_ae_last120_non3digit_month"  + m + ".pkl", 'wb'), protocol=4)


/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (3,5,6,7,13,14,15,17,20,21,22,26,27,28,30,31,34,35,37,40,41,42,43,45,46,47,48,49,53,54,55,56,61,62,63,68,69,70,73,74,75,76,77,79,81,82,83,84,87,89,90,91,96,97,98,99,103,104,105,108,110,111,112,115,116,117,118,119,120,121,124,125,126,128,129,132,133,135,136,137,138,139,140,143,145,146,147,148,149,151,152,153,154,155,156,157,159,160,161,162,165,166,167,168,171,172,174,175,176,177,178,180,181,182,184,185,186,187,188,189,190,191,192,194,195,196,200,201,202,203,204,205,207,208,209,210,216,217,218,223,224,230,231,237,238,244,245,246,252,259,266,267,273,275,279,280,282,283,285,286,287,290,291,293,294,296,297,298,299,300,301,306,307,308,310,311,313,314,315,318,320,321,322,324,325,327,328,329,330,331,332,333,334,335,336,342,343,345,346,347,348,349,350,351,352,354,355,356,357,358,359,360,361,363,364,365,366) have mixed types. Specify dtype option on import or set low

month size:  2 730190052
used days:  119 0
total size before:  (1886796, 120)
New dataset created
sequence length:  1478943
empty events with nonzero labels:  8265
time:  839.4235293865204


/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,30,31,32,33,34,35,36,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,86,87,88,89,90,91,92,94,95,97,98,100,101,102,103,104,105,106,107,108,109,110,111,112,113,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,206,207,208,209,210,211,212,213,214,215,216,217,218,220,221,222,223,224,225,226,227,228,229,230,231,232,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,

month size:  3 724429944
used days:  119 0
total size before:  (1871912, 120)
New dataset created
sequence length:  1454134
empty events with nonzero labels:  8709
time:  874.7379584312439


/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (3,4,10,11,18,24,25,31,32,38,39,44,45,46,51,52,53,58,59,60,61,65,66,67,68,73,74,80,81,87,88,92,94,95,96,101,102,108,109,111,115,116,121,122,123,129,130,131,135,136,137,142,143,144,145,150,151,157,158,159,164,165,171,172,179,185,186,200,207,213,214,219,220,221,227,228,231,234,235,239,240,241,242,246,248,249,255,256,260,261,262,263,267,268,269,270,275,276,277,278,283,284,286,287,289,290,291,296,297,298,304,305,307,308,311,312,315,316,318,319,321,324,325,326,330,331,332,333,337,339,340,341,344,345,346,347,352,353,354,355,360,361,366) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


...

month size:  9 688346838
used days:  119 0
total size before:  (1778674, 120)
New dataset created
sequence length:  1461131
empty events with nonzero labels:  7419
time:  288.7220561504364


/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (17,24,31,38,45,51,52,56,57,58,59,65,66,72,73,79,80,85,86,87,92,93,94,100,101,107,108,114,115,121,122,128,129,135,136,142,143,144,149,150,156,157,164,170,171,177,178,184,185,191,192,198,199,205,206,212,213,217,219,220,226,227,233,234,240,241,242,247,248,254,255,261,262,268,269,276,277,283,289,290,296,297,303,304,310,311,317,318,324,325,331,332,338,339,340,345,346,353,360) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


month size:  10 679187709
used days:  119 0
total size before:  (1755007, 120)
New dataset created
sequence length:  1476727
empty events with nonzero labels:  7492
time:  279.44660115242004


/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (7,13,14,21,25,26,27,28,35,42,48,49,54,55,56,62,63,69,70,76,77,83,84,90,91,97,98,103,104,105,111,112,119,125,126,133,139,140,146,147,153,154,160,161,167,168,174,175,181,182,188,189,195,196,202,203,208,209,210,211,224,230,231,236,237,238,244,245,246,251,252,258,259,265,266,272,273,280,286,287,293,294,300,301,308,314,322,335,342,343,357,364) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


month size:  11 668919051
used days:  119 0
total size before:  (1728473, 120)
New dataset created
sequence length:  1508506
empty events with nonzero labels:  6336
time:  279.8270537853241


## Random things to check

In [90]:
def get_count(df, vocab, feat_colnames, label_colnames, day_length=30, max_length=50, mode='eval', keep_length=6):
    # create dataset
    start_time = time.time()
    print("used days: ", feat_colnames[-day_length], feat_colnames[-1])
    data = df[feat_colnames[-day_length:]].to_numpy()
    sequence = []
    print("total size before: ", data.shape)
    max_count = 0
    max_day = 0
    days = 0
    mid_days = 0
    day_dict = {}
    labels = df[label_colnames].to_numpy()
    for i in range(len(data)):
        sentence = []
        if not labels[i].any():
            continue
        mask = []
        count = 0
        
        for j in range(len(data[i])):
            
            words = str(data[i][j]).replace('d_s', 'd_')
            if words == 'nan':
                continue
            words = words.split(', ')
            
            if len(words) > max_length:
                days += 1
            if len(words) > max_length - 5:
                mid_days += 1
            max_day = max(max_day, len(words))
            #print(max_day)
            count += len(words)
            '''words = sorted([vocab.stoi[w] if w in vocab.stoi else vocab.stoi['<unk>'] for w in words])
            
            if len(words) > max_length:
                words = words[:max_length]
            
            words = words + [vocab.stoi['<pad>']] * (max_length - len(words))
                    
            sentence.append(words)'''
        max_count = max(max_count, count)
        #if count == 0:
           # print("empty: ", i)
        if count in day_dict:
            day_dict[count] += 1
        else:
            day_dict[count] = 1
    print(max_count, max_day, days, mid_days)
    return day_dict

In [91]:
day_dict = get_count(np_ae, vocab, x_lst, y_lst, day_length=120, max_length=30)

used days:  119 0
total size before:  (1903423, 120)
1118 65 588 2134


In [140]:
row = random.randint(0, 1562223)
print(whole_dataset[3][row][:5])
print(whole_dataset[1][row][:5])

[[1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0.]]
[[  28   83 1182    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0]
 [   2    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0]
 [   2    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0]
 [   2    0    0    0    0    0    0    0    0    0  

In [77]:
! aws s3 cp s3://cmsai-mrk-amzn/CSVModelInputs/miguel/ae_patients_365_20110101_wo_0s.csv data/

download: s3://cmsai-mrk-amzn/CSVModelInputs/miguel/ae_patients_365_20110101_wo_0s.csv to data/ae_patients_365_20110101_wo_0s.csv
